<img src="https://i.ibb.co/qjt4Ymb/2022-09-19-004719.png" alt="2022-09-19-004719" border="0">

# Topic 15: CNN with PyTorch

# Kissipo-CNN-FashionMNIST
FashionMNIST
![FashionMNIST](https://repository-images.githubusercontent.com/551880225/63622c63-717c-4b14-8a18-d750e395f484)

## Load Libraries

In [ ]:
import torch
print(torch.__version__)

1.13.1+cu116


If the GPU is available use it for the computation otherwise use the CPU.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Part-1 The training phase

In [ ]:
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## IPO-M: Building a DL model (Sequential API, Graph mode)
$M(x)=\hat{y}$

In [ ]:
#Building a DL model 
class NeuralNetwork(nn.Module):
    def __init__(self):
      super().__init__()
      # define layers
      self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
      self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

      self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
      self.fc2 = nn.Linear(in_features=120, out_features=60)
      self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
      # conv 1
      t = self.conv1(t)
      t = F.relu(t)
      t = F.max_pool2d(t, kernel_size=2, stride=2)

      # conv 2
      t = self.conv2(t)
      t = F.relu(t)
      t = F.max_pool2d(t, kernel_size=2, stride=2)

      # fc1
      t = t.reshape(-1, 12*4*4)
      t = self.fc1(t)
      t = F.relu(t)

      # fc2
      t = self.fc2(t)
      t = F.relu(t)

      # output
      t = self.out(t)
      # don't need softmax here since we'll use cross-entropy as activation.

      return t

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)


## IPO-I: input data

### Load FashionMNIST dataset

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## IPO-P: Training model

### Choose an optimizer and loss function for model building 


In [ ]:
#Create a loss function.
loss_fn = nn.CrossEntropyLoss()

In [ ]:
#Create an optimizer with the desired parameters.
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

## Training model 

In [ ]:
#model.fit(x_train, y_train, epochs=10)
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.311429  [    0/60000]
loss: 2.305575  [ 6400/60000]
loss: 2.308418  [12800/60000]
loss: 2.302438  [19200/60000]
loss: 2.317723  [25600/60000]
loss: 2.306368  [32000/60000]
loss: 2.296357  [38400/60000]
loss: 2.305718  [44800/60000]
loss: 2.308988  [51200/60000]
loss: 2.288970  [57600/60000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.304070 

Epoch 2
-------------------------------
loss: 2.308783  [    0/60000]
loss: 2.303326  [ 6400/60000]
loss: 2.305264  [12800/60000]
loss: 2.300435  [19200/60000]
loss: 2.314563  [25600/60000]
loss: 2.303186  [32000/60000]
loss: 2.295939  [38400/60000]
loss: 2.303321  [44800/60000]
loss: 2.306543  [51200/60000]
loss: 2.288968  [57600/60000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302126 

Epoch 3
-------------------------------
loss: 2.306682  [    0/60000]
loss: 2.301736  [ 6400/60000]
loss: 2.302920  [12800/60000]
loss: 2.298892  [19200/60000]
loss: 2.311773  [25600/60000]
loss: 2.300995  [32000/600